# Intermediate Machine Learning
## Contents
1. [Dealing with missing values](http://localhost:8888/notebooks/Documents/GitHub/Kaggle/home-data-for-ml-course/Intermediate%20Machine%20Learning.ipynb#Dealing-with-missing-values)
2. [Dealing with categorical values](http://localhost:8888/notebooks/Documents/GitHub/Kaggle/home-data-for-ml-course/Intermediate%20Machine%20Learning.ipynb#Dealing-with-categorical-values)

## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

## Loading train and test data
We use the data from home-data-for-ml-course from Kaggle:
* train.csv, which we split into train and validation datasets
* test.csv, which we use to test our model

We preliminarily clean the data, by:
* dropping the rows with a null ('Nan') target variable
* separating the target variable (to y) and the predictor variables (to X)

In [122]:
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id') # has no target column ('SalePrice')

# Drop rows (axis=0) without the target predictor, where SalePrice = NaN
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)

# Separate the target variable (to the variable y), then remove target column (axis=1) from the predictor variables
y = X_full.SalePrice
X = X_full.drop(axis=1, columns=['SalePrice'])

For the purposes of exploring how to deal with missing values, we define X and X_test to exclude categorical predictors.

In [48]:
# To keep things simple, only use numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Separate data into train and validation datasets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Dealing with missing values

3 methods:

1. Dropping columns
2. Imputing columns (strategy='mean', 'median', 'most_frequent' or 'constant') - see https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html
3. Imputing columns and adding a col_was_missing column with DataFrame.isnull()

## Preliminarily investigating the data

In [4]:
# There are some columns with missing values
X_train.head(10)

MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
Id                                                                            
619           20         90.0    11694            9            5       2007   
871           20         60.0     6600            5            5       1962   
93            30         80.0    13360            5            7       1921   
818           20          NaN    13265            8            5       2002   
303           20        118.0    13704            7            5       2001   
1455          20         62.0     7500            7            5       2004   
41            20         84.0     8658            6            5       1965   
960          160         24.0     2572            7            5       1999   
76           180         21.0     1596            4            5       1973   
1390          50         60.0     6000            6            6       1941   

      YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  ...  GarageArea  \
Id                                                      ...               
619           2007       452.0          48           0  ...         774   
871           1962         0.0           0           0  ...         308   
93            2006         0.0         713           0  ...         432   
818           2002       148.0        1218           0  ...         857   
303           2002       150.0           0           0  ...         843   
1455          2005         0.0         410           0  ...         400   
41            1965       101.0         643           0  ...         440   
960           1999         0.0         604           0  ...         484   
76            1973         0.0         462           0  ...         297   
1390          1950         0.0         375           0  ...         440   

      WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  \
Id                                                                     
619            0          108              0          0          260   
871            0            0              0          0            0   
93             0            0             44          0            0   
818          150           59              0          0            0   
303          468           81              0          0            0   
1455           0          113              0          0            0   
41             0          138              0          0            0   
960            0           44              0          0            0   
76           120          101              0          0            0   
1390           0            0              0          0            0   

      PoolArea  MiscVal  MoSold  YrSold  
Id                                       
619          0        0       7    2007  
871          0        0       8    2009  
93           0        0       8    2009  
818          0        0       7    2008  
303          0        0       1    2006  
1455         0        0      10    2009  
41           0        0      12    2006  
960          0        0       5    2010  
76           0        0      11    2009  
1390         0        0       3    2007  

[10 rows x 36 columns]

In [5]:
# Gives (rows, columns) of the training and validation datasets
print('Shape of training dataset:', X_train.shape)
print('Shape of validation dataset:', X_valid.shape)

Shape of training dataset: (1168, 36)
Shape of validation dataset: (292, 36)


In [6]:
# Identifies number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [7]:
# Creates a list ('cols_with_missing') containing the headers of columns with missing values

cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
print(cols_with_missing)

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']


## Creating a function to score datasets
* In this notebook, we use the 3 methods above (dropping columns, imputation, and imputation with adding a _was_dropped column)
* The 3 methods modify the X_train and X_valid datasets
* We score the ability of the 3 modified X_train datasets to train a model; against its performance in the modified X_valid datasets

In [8]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    pred = model.predict(X_valid)
    return mean_absolute_error(pred, y_valid)

## Method 1: Dropping columns
Use the method to drop columns with missing values, i.e. NaN or None.
    
    [DataFrameName].drop(cols_with_missing, axis=1)
    
This needs to be applied to both the X_train and X_valid datasets

In [9]:
# Make a copy of the training and validation datasets
X_train_drop = X_train.copy()
X_valid_drop = X_valid.copy()

# Drop the columns in cols_with_missing
X_train_drop.drop(cols_with_missing, axis=1, inplace=True)
X_valid_drop.drop(cols_with_missing, axis=1, inplace=True)

print('Shape of training dataset, after dropping columns:', X_train_drop.shape)
print('Shape of validation dataset, after dropping columns:', X_valid_drop.shape)

Shape of training dataset, after dropping columns: (1168, 33)
Shape of validation dataset, after dropping columns: (292, 33)


## Method 2: Imputation
Imputation refers to replacing NaN values with the:
* the mean (strategy='mean')
* the median (strategy='median')
* the mode (strategy='most_frequent')
* a constant value (strategy='constant', fill_value=k)

See 

    help(SimpleImputer())

for more information.

In [10]:
# Make a copy of the training and validation datasets
X_train_imput1 = X_train.copy()
X_valid_imput1 = X_valid.copy()

# Check whether there are any missing values in X_train and X_valid
print(X_train.isnull().any().any())
print(X_valid.isnull().any().any())

# Create an instance of SimpleImputer
my_imputer_1 = SimpleImputer()

X_train_imput1 = pd.DataFrame(my_imputer_1.fit_transform(X_train_imput1))
X_valid_imput1 = pd.DataFrame(my_imputer_1.transform(X_valid_imput1))

# Check whether there are an missing values in X_train and X_valid
print(X_train_imput1.isnull().any().any())
print(X_valid_imput1.isnull().any().any())
print()
print('Shape of training dataset, after imputing values:', X_train_imput1.shape)
print('Shape of validation dataset, after imputing values:', X_valid_imput1.shape)

# Imputation removes column headings, add them back
X_train_imput1.columns = X_valid.columns
X_valid_imput1.columns = X_valid.columns

True
True
False
False

Shape of training dataset, after imputing values: (1168, 36)
Shape of validation dataset, after imputing values: (292, 36)


## Method 3: Imputation redux (adding another column)
To help the model distinguish between actual and imputed values, for each column that has imputed values, we add another column ('ColName_was_missing') that contains X_train[ColName].isnull() or X_valid[ColName].isnull().

In [11]:
# Make a copy of the training and validation datasets
X_train_imput2 = X_train.copy()
X_valid_imput2 = X_valid.copy()

# Add columns to label rows with imputed values
for col in cols_with_missing:
    X_train_imput2[col + '_was_missing'] = X_train_imput2[col].isnull()
    X_valid_imput2[col + '_was_missing'] = X_valid_imput2[col].isnull()
    
# Inspect dataframe to see that rows are added correctly
# X_train_imput2.head()
# X_valid_imput2.head()

# Impute missing values, as before. Another instance of SimpleImputer() is created again, since it needs to be retrained
my_imputer_2 = SimpleImputer()
X_train_imput2 = pd.DataFrame(my_imputer_2.fit_transform(X_train_imput2))
X_valid_imput2 = pd.DataFrame(my_imputer_2.transform(X_valid_imput2))

# Check whether there are an missing values in X_train and X_valid
print(X_train_imput2.isnull().any().any())
print(X_valid_imput2.isnull().any().any())
print()
print('Shape of training dataset, after imputing values and adding columns:', X_train_imput2.shape)
print('Shape of validation dataset, after imputing values and adding columns:', X_valid_imput2.shape)

False
False

Shape of training dataset, after imputing values and adding columns: (1168, 39)
Shape of validation dataset, after imputing values and adding columns: (292, 39)


## Testing the interventions of the model
* As defined in the score_dataset() function above, the model is a RandomForestRegressor

In [12]:
# help(RandomForestRegressor)

# In this case, score_dataset(X_train, X_valid, y_train, y_valid) would return an error.

print('Method 1 - drop columns:', round(score_dataset(X_train_drop, X_valid_drop, y_train, y_valid), 2))
print('Method 2 - impute values:', round(score_dataset(X_train_imput1, X_valid_imput1, y_train, y_valid), 2))
print('Method 3 - impute values, add isnull column:', round(score_dataset(X_train_imput2, X_valid_imput2, y_train, y_valid), 2))

Method 1 - drop columns: 17837.83
Method 2 - impute values: 18062.89
Method 3 - impute values, add isnull column: 18148.42


# Dealing with categorical values
3 methods:
1. Dropping categorical values
2. Assigning a value (only works for ordinal categories, like Likert scale <==> [-2, -1, 0, 1, 2]
3. One-hot encoding (but doesn't work if there are too many categories for each predictor variable)

## Some pre-processing
We need to respecify X and X_test, since we'd excluded categorical predictor variables earlier.
* Redefine X and X_test from X_full and X_test_full, including categorical predictor variables
* Call train_test_split on X, y again

In [59]:
# Call train_test_split on X, y
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, train_size = 0.8, test_size = 0.2, random_state=0)

# Remove columns with NaN values
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()]
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)
# print(list(X_valid_full.columns))

# Select categorical columns with low cardinality
low_cardinality_cols = [col for col in X_train_full.columns 
                        if X_train_full[col].dtypes == 'object' and X_train_full[col].nunique() < 10
                       ]
# print(low_cardinality_cols)

# Select numerical columns
num_cols = [col for col in X_train_full.columns if X_train_full[col].dtypes in ['int64', 'float64']]
# print(num_cols)

# Keep selected columns only
my_cols = low_cardinality_cols + num_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']


## Method 1: Dropping categorical values

In [56]:
# Make a copy of the training and validation datasets
drop_X_train = X_train.copy()
drop_X_valid = X_valid.copy()

# Drop all categorical variables from the datasets
drop_X_train = drop_X_train.select_dtypes(exclude='object')
drop_X_valid = drop_X_valid.select_dtypes(exclude='object')

# Run a check on dtypes of drop_X_train and drop_X_valid
# drop_X_train.dtypes == 'object'
# drop_X_valid.dtypes == 'object'

## Method 2: Assigning a numerical value to each categorical class
The main challenge is that some categorical classes may only be present in the validation dataset and not in the training dataset:

In [68]:
# There are classes in one dataset that are not present in the other
object_cols = [col for col in X_train.columns if X_train[col].dtypes == 'object']
print(set(X_train[object_cols[12]]))
print(set(X_valid[object_cols[12]]))
set(X_train[X_train.columns[15]]) == set(X_valid[X_train.columns[15]])

{'Tar&Grv', 'Metal', 'WdShngl', 'Membran', 'CompShg', 'Roll', 'WdShake'}
{'Tar&Grv', 'ClyTile', 'WdShngl', 'CompShg'}


False

In [69]:
# Identifying all categorical values
# X_train.dtypes is a dataframe with the column headers as the index (left), and the dtype as the other column (right)
# X_train.dtypes == 'object' is a dataframe with the column headers as the index (left), and a bool as the other column (right)
s = (X_train.dtypes == 'object')

# s[s] is equal to s[s == True]; returns the second dataframe, but only for values where the bool == True
# print(s[s])
# s[s].index returns an Index() of all indexes in s[s]
object_cols = list(s[s].index)
print('Categorical variables:\n\n', object_cols)

Categorical variables:

 ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']


In [70]:
# Make a copy of the training and validation datasets
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Only select categorical variables where the classes in the training dataset == classes in the validation dataset
good_label_cols = [col for col in object_cols if set(X_valid[col]) == set(X_train[col])]

# Select problematic columns to be removed
bad_label_cols = list(set(object_cols) - set(good_label_cols))

print('Categorical labels that will be label encoded:\n', good_label_cols)
print('\nCagegorical labels that will be dropped:\n', bad_label_cols)

Categorical labels that will be label encoded:
 ['MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'BldgType', 'HouseStyle', 'ExterQual', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleCondition']

Cagegorical labels that will be dropped:
 ['RoofMatl', 'ExterCond', 'Foundation', 'HeatingQC', 'Functional', 'Condition1', 'Heating', 'RoofStyle', 'LandSlope', 'Condition2', 'SaleType', 'Utilities']


In [75]:
# Drop problematic columns from X_train and X_valid
label_X_train.drop(bad_label_cols, axis=1, inplace=True)
label_X_valid.drop(bad_label_cols, axis=1, inplace=True)

In [85]:
# Create an instance of LabelEncoder
label_encoder = LabelEncoder()
for col in good_label_cols:
    label_X_train[col] = label_encoder.fit_transform(label_X_train[col])
    label_X_valid[col] = label_encoder.transform(label_X_valid[col])
    
# Check that there are no dtypes = 'object'
# s = label_X_train.dtypes == 'object'
# s.any()

## Investigating cardinality

In [90]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col : X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x : x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9)]

## Method 3: One-hot encoding

In [113]:
# Make a copy of training and validation datasets
OH_X_train = X_train.copy()
OH_X_valid = X_valid.copy()

# Creating an instance of OneHotEncoder
OH_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# One-hot encoding
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(OH_X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(OH_X_valid[object_cols]))

# One-hot encoding removes indexes; add them back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns from OH_X_train, OH_X_valid
OH_X_train.drop(object_cols, axis=1, inplace=True)
OH_X_valid.drop(object_cols, axis=1, inplace=True)

# Combine OH_X_train/valid with OH_cols_train/valid
OH_X_train = pd.concat([OH_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([OH_X_valid, OH_cols_valid], axis=1)

# None of the remaining dtypes are objects
# s = OH_X_train.dtypes == 'object'
# s[s]

## Testing the interventions of the model

In [114]:
print("MAE from Approach 1 (Drop columns with categorical variables):") 
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))
print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with categorical variables):
17837.82570776256
MAE from Approach 2 (Label Encoding):
17588.240936073056
MAE from Approach 3 (One-Hot Encoding):
17525.345719178084
